In [1]:
import sys
sys.path.append("../..")

In [2]:
from copy import deepcopy

In [3]:
import dvc.api
import pandas as pd
import torch
import numpy as np

In [4]:
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [5]:
import yaml

In [6]:
md = yaml.safe_load(open('/workspaces/query_understanding_model/models/product_title_multitask_multimodal/version_1/config.yaml', 'r')
)['model']['multitask_specs_dict']

In [7]:
md = {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]},
  'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'emb_singlemodal_wishquery2googletitle': None,
 'clm_singlemodal_alititle2v121tax': None,
 'clm_singlemodal_wishtitle2pseudov121tax': None,
 'dlm_singlemodal_wishtitle': None,
 'emb_singlemodal_wishtitle2pseudov121tax': None,
 'emb_singlemodal_alititle2v121tax': None,
 'seqclf_singlemodal_alititle2v121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'seqclf_singlemodal_wishtitle2pseudov121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}}}

In [9]:
model = LLM_MultitaskMultimodal.load_from_checkpoint(
    '/workspaces/query_understanding_model/models/product_title_multitask_multimodal/version_1/epoch=0-step=45000.ckpt',
    multitask_specs_dict = md
)

INFO:root:Unused kwargs when getting t5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}, 'specs': {'clf_head': 'clf_head', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_type': 'taxonomy'}}, 'emb_singlemodal_wishquery2googletitle': None, 'clm_singlemodal_alititle2v121tax': None, 'clm_singlemodal_wishtitle2pseudov121tax': None, '

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
import torch 
torch.cuda.device_count()

1

In [12]:
label_map = {}
label_map_file = '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt'
with open(label_map_file, 'r') as f:
    for l in f:
        if len(l):
            label_map[l.replace("\n", "")] = len(label_map)

In [13]:
df_clipmore_test = pd.read_json(dvc.api.get_url(
    "data/wish_clipmore/Wish_Clipmore_Tahoe_Test_Dedup.json",
    repo='git@github.com:junwang-wish/query_understanding_data.git'
), lines=True)

INFO:aiobotocore.credentials:Found credentials in environment variables.


In [14]:
model.cuda()
model.eval()
tmp = 2

# demo some tasks used to train multitask v0.2

In [15]:
def mm_emb(item):
    assert ('query' in item and len(item) == 1) or ('title' in item and len(item) == 1) or \
        ('img_embedding' in item and len(item) == 1) or \
        ('title' in item and 'img_embedding' in item and len(item) == 2)
    if 'query' in item:
        inputs = model.tokenizer(f"Embed query: {item['query']}", return_tensors='pt')
        hidden_states = model.get_hidden_states(
            input_ids = inputs['input_ids'].to(model.device), 
            attention_mask = inputs['attention_mask'].to(model.device)
        )
        return hidden_states.detach()
    elif 'title' in item and 'img_embedding' in item:
        inputs = model.tokenizer(f"Embed product with image: [title start] {item['title']} [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
        input_ids = inputs['input_ids'].to(model.device)
        img_embedding = torch.tensor([eval(item['img_embedding'])]).to(model.device)
        proj_img_embedding = model.proj_head(img_embedding)
        embs_swapped = torch.where(
            (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
            torch.zeros_like(proj_img_embedding.unsqueeze(1)),
            model.transformer.get_input_embeddings()(input_ids)
        )
        attention_mask = inputs['attention_mask'].to(model.device)
        hidden_states = model.get_hidden_states(
            input_ids = input_ids.to(model.device), 
            attention_mask = attention_mask.to(model.device),
            inputs_embeds = embs_swapped 
        )
        return hidden_states.detach()
    elif 'title' in item:
        inputs = model.tokenizer(f"Embed product: {item['title']}", return_tensors='pt')
        hidden_states = model.get_hidden_states(
            input_ids = inputs['input_ids'].to(model.device), 
            attention_mask = inputs['attention_mask'].to(model.device)
        )
        return hidden_states.detach()
    elif 'img_embedding' in item:
        inputs = model.tokenizer("Embed product with image: [title start] [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
        input_ids = inputs['input_ids'].to(model.device)
        img_embedding = torch.tensor([eval(item['img_embedding'])]).to(model.device)
        proj_img_embedding = model.proj_head(img_embedding)
        embs_swapped = torch.where(
            (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
            torch.zeros_like(proj_img_embedding.unsqueeze(1)),
            model.transformer.get_input_embeddings()(input_ids)
        )
        attention_mask = inputs['attention_mask'].to(model.device)
        hidden_states = model.get_hidden_states(
            input_ids = input_ids.to(model.device), 
            attention_mask = attention_mask.to(model.device),
            inputs_embeds = embs_swapped 
        )
        return hidden_states.detach()
    else:
        raise Exception('invalid item')

In [16]:
from tqdm import tqdm

In [17]:
recs = []
for i in tqdm(df_clipmore_test.to_dict('records')):
    i['title_embedding'] = mm_emb({'title': i['title']}).cpu().tolist()[0]
    i['titleimg_embedding'] = mm_emb({'title': i['title'], 'img_embedding': i['img_embedding']}).cpu().tolist()[0]
    i['nullimg_embedding'] = mm_emb({'img_embedding': i['img_embedding']}).cpu().tolist()[0]
    recs.append(deepcopy(i))

100%|██████████| 147/147 [00:28<00:00,  5.18it/s]


In [18]:
df_clipmore_test_emb = pd.DataFrame(recs)

In [32]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize

In [93]:
hidden_states1 = normalize(np.array(df_clipmore_test_emb.img_embedding.apply(eval).tolist()))
hidden_states2 = normalize(np.array(df_clipmore_test_emb.title_embedding.tolist()))
hidden_states3 = normalize(np.array(df_clipmore_test_emb.nullimg_embedding.tolist()))
hidden_states4 = normalize(np.array(df_clipmore_test_emb.titleimg_embedding.tolist()))
hidden_states5 = normalize((np.array(df_clipmore_test_emb.title_embedding.tolist()) + \
    np.array(df_clipmore_test_emb.img_embedding.apply(eval).tolist()))/2)
# lab = df_clipmore_test_emb['v121_category'].apply(lambda x: tuple(x))
lab = df_clipmore_test_emb['v121_category'].apply(lambda x: x[0])

In [94]:
silhouette_score(hidden_states1, lab), silhouette_score(hidden_states2, lab), \
    silhouette_score(hidden_states3, lab), silhouette_score(hidden_states4, lab), \
    silhouette_score(hidden_states5, lab)

(-0.0667835589327277,
 -0.014445437032401557,
 0.0,
 -0.025193233387842528,
 -0.015496081315965872)

In [95]:
import umap.plot
import umap
import numpy as np
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20

hidden_states = np.array(df_clipmore_test_emb.img_embedding.apply(eval).tolist())
# hidden_states = np.array(df_clipmore_test_emb.title_embedding.tolist())
# hidden_states = np.array(df_clipmore_test_emb.nullimg_embedding.tolist())
# hidden_states = np.array(df_clipmore_test_emb.titleimg_embedding.tolist())

mapper = umap.UMAP().fit(hidden_states)
proj_data = mapper.transform(hidden_states)


output_file("toolbar.html")

source = ColumnDataSource(
        data=dict(
            x=proj_data[:,0],
            y=proj_data[:,1],
            desc=df_clipmore_test_emb['title'].tolist(),
            cat=df_clipmore_test_emb['v121_category'].apply(lambda x: " > ".join(x)).tolist(),
            cat_zero=df_clipmore_test_emb['v121_category'].apply(lambda x: x[0]).tolist(),
            imgs = df_clipmore_test_emb['product_id'].apply(lambda x: f"https://canary.contestimg.wish.com/api/webimage/{x}-large.jpg").tolist()
        )
    )

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" height="100" alt="@imgs" width="100"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@desc</span>
                <span style="font-size: 17px; font-weight: bold;">>>>></span>
                <span style="font-size: 17px; font-weight: bold;">@cat</span>
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
    )

p = figure(plot_width=1200, plot_height=800, tools=[hover],
           title="Mouse over the dots")
cat0 = list(set(df_clipmore_test_emb['v121_category'].apply(lambda x: x[0]).tolist()))
p.circle('x', 'y', size=5, color=factor_cmap('cat_zero', palette=Category20[20], factors=cat0), source=source)

show(p)


In [17]:
# df_clipmore_test[['product_id', 'title', 'v121_category']].sample(1).to_dict('records')

[{'product_id': '6180a2722b4fee63052c220c',
  'title': 'Trend women fashion skirt newest Friday the 13th Christmas Designs 3d print fashion long sleeve teenagers casual home pajamas dress Y10',
  'v121_category': ['mother & kids',
   "girls' clothing",
   'sleepwear & robes',
   'nightgowns']}]

In [14]:
img_url = "https://canary.contestimg.wish.com/api/webimage/6180a2722b4fee63052c220c-large.jpg"

In [15]:
img_embedding = torch.tensor([eval(df_clipmore_test[df_clipmore_test.product_id == "6180a2722b4fee63052c220c"][
    ['img_embedding']].to_dict('records')[0]['img_embedding'])])

In [16]:
proj_img_embedding = model.proj_head(img_embedding.to(model.device))

In [17]:
(img_embedding**2).sum(), (proj_img_embedding**2).sum()

(tensor(342.6460), tensor(104.1131, device='cuda:0', grad_fn=<SumBackward0>))

In [18]:
# multimodal clm-based title generation with original clip emb (result is gibberish)
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Generate title for product with image: [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    img_embedding.unsqueeze(1).to(model.device),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        inputs_embeds=embs_swapped, attention_mask=attention_mask,
        length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()


(['The New York City Poster Print by Panoramic Images',
  'The Last Air Force Movie Poster Print (27 x 40) - Item # MOV1938',
  'The Last Air Force Movie Poster Print (27 x 40) - Item # MOV193838'],
 tensor([1.1393e-07, 4.8516e-11, 4.7726e-11], device='cuda:0'),
 tensor(1.1403e-07, device='cuda:0'))

In [19]:
(embs_swapped[0]**2).sum(-1) # the norm of unprojected img embedding is similar to text (kinda strange..?)

tensor([328.7288, 345.5509, 375.6101, 360.8777, 329.4005, 373.5842, 355.2832,
        365.4133, 389.5612, 344.4916, 359.6084, 365.2524, 342.6460, 389.5612,
        344.4916, 351.9210, 365.2524, 338.8931], device='cuda:0',
       grad_fn=<SumBackward1>)

In [20]:
# multimodal clm-based title generation with projected clip emb (result is good)
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Generate title for product with image: [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    proj_img_embedding.unsqueeze(1).to(model.device),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        inputs_embeds=embs_swapped, attention_mask=attention_mask,
        length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()


(["New Fashion Women/Men's 3D Print Christmas Hoodies/hooded Sweatshirts + Pants Suit Clothes",
  "New Fashion Women/Men's 3D Print Christmas Hoodies/hooded Sweatshirts + Pants Suits",
  "New Fashion Women/Men's 3D Print Christmas Hoodies/hooded Sweatshirts + Pants Trousers Suit Clothes"],
 tensor([4.2894e-06, 3.9345e-07, 3.4168e-07], device='cuda:0'),
 tensor(5.0245e-06, device='cuda:0'))

In [21]:
(embs_swapped[0]**2).sum(-1) # strange that the norm of projected img embedding is much lower than text, is it normal?

tensor([328.7288, 345.5509, 375.6101, 360.8777, 329.4005, 373.5842, 355.2832,
        365.4133, 389.5612, 344.4916, 359.6084, 365.2524, 104.1131, 389.5612,
        344.4916, 351.9210, 365.2524, 338.8931], device='cuda:0',
       grad_fn=<SumBackward1>)

In [22]:
# multimodal dlm-based title denoising
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Denoise product with image: [title start] <extra_id_0> [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    proj_img_embedding.unsqueeze(1),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        inputs_embeds=embs_swapped, attention_mask=attention_mask,
        length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()



(['Christmas Hoodie',
  "New Fashion Women/Men's 3D Print Christmas Hoodies/hooded Sweatshirts",
  "New Fashion Women/Men's 3D Print Christmas Hoodies/hooded Sweatshirts + Pants Suit Clothes"],
 tensor([1.6350e-03, 4.1582e-06, 9.6984e-07], device='cuda:0'),
 tensor(0.0016, device='cuda:0'))

In [23]:
# multimodal dlm-based title denoising with image zeroed
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Denoise product with image: [title start] <extra_id_0> [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    torch.zeros_like(proj_img_embedding.unsqueeze(1)),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
model.tokenizer.batch_decode(
    model.transformer.generate(
            inputs_embeds=embs_swapped, attention_mask=attention_mask,
            length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
            do_sample=False)
)


['<pad><extra_id_0> X-Mas<extra_id_1></s><pad><pad><pad><pad><pad><pad>',
 '<pad><extra_id_0> X-Mas -<extra_id_1></s><pad><pad><pad><pad>',
 '<pad><extra_id_0> X-Mas X-Mas<extra_id_1></s>']

In [24]:
# multimodal seqclf-based categorize product
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Classify product with image: [title start] Trend women fashion skirt newest Friday the 13th Christmas Designs 3d print fashion long sleeve teenagers casual home pajamas dress Y10 [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    proj_img_embedding.unsqueeze(1),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
hidden_states = model.get_hidden_states(
    input_ids = input_ids.to(model.device), 
    attention_mask = attention_mask.to(model.device),
    inputs_embeds = embs_swapped 
)

logits = model.clf_head(hidden_states)
prediction = logits
probs = prediction.sigmoid()
top_probs, top_pred_indices = probs.topk(probs.size(1))
assert prediction.size(1) == len(label_map)

[(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)[:10]]


[("women's clothing", 0.9481745362281799),
 ("women's clothing > underwear & sleepwear > women's sleepwear",
  0.22060589492321014),
 ("women's clothing > matching sets", 0.11320054531097412),
 ("women's clothing > underwear & sleepwear", 0.07634633034467697),
 ('mother & kids', 0.06752103567123413),
 ("women's clothing > underwear & sleepwear > women's sleepwear > pajama sets",
  0.032819561660289764),
 ("women's clothing > bottoms", 0.02738446369767189),
 ('mother & kids > matching family outfits', 0.027149299159646034),
 ("women's clothing > matching sets > skirt sets", 0.023866329342126846),
 ("women's clothing > underwear & sleepwear > women's sleepwear > sleep tops",
  0.01636957749724388)]

In [25]:
# multimodal seqclf-based categorize product with image zeroed
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Classify product with image: [title start] Trend women fashion skirt newest Friday the 13th Christmas Designs 3d print fashion long sleeve teenagers casual home pajamas dress Y10 [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    torch.zeros_like(proj_img_embedding.unsqueeze(1)),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
hidden_states = model.get_hidden_states(
    input_ids = input_ids.to(model.device), 
    attention_mask = attention_mask.to(model.device),
    inputs_embeds = embs_swapped 
)

logits = model.clf_head(hidden_states)
prediction = logits
probs = prediction.sigmoid()
top_probs, top_pred_indices = probs.topk(probs.size(1))
assert prediction.size(1) == len(label_map)

[(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)[:10]]


[("women's clothing", 0.9443934559822083),
 ("women's clothing > underwear & sleepwear > women's sleepwear",
  0.24849344789981842),
 ("women's clothing > underwear & sleepwear", 0.11866361647844315),
 ("women's clothing > matching sets", 0.08950916677713394),
 ('mother & kids', 0.057061802595853806),
 ("women's clothing > bottoms", 0.05022074282169342),
 ("women's clothing > matching sets > skirt sets", 0.033552687615156174),
 ('home & garden', 0.021680690348148346),
 ("women's clothing > underwear & sleepwear > women's sleepwear > pajama sets",
  0.02051149122416973),
 ('mother & kids > matching family outfits', 0.016158943995833397)]

In [26]:
# multimodal clm-based categorize product
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Generate taxonomy for product with image: [title start] Trend women fashion skirt newest Friday the 13th Christmas Designs 3d print fashion long sleeve teenagers casual home pajamas dress Y10 [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    proj_img_embedding.unsqueeze(1),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        inputs_embeds=embs_swapped, attention_mask=attention_mask,
        length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()

(["women's clothing > matching sets > skirt sets",
  'mother & kids > pregnancy & maternity > maternity clothing > sleep & lounge',
  "women's clothing > matching sets > dress sets"],
 tensor([0.6870, 0.0868, 0.0785], device='cuda:0'),
 tensor(0.8523, device='cuda:0'))

In [27]:
# multimodal clm-based categorize product with image zeroed
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Generate taxonomy for product with image: [title start] Trend women fashion skirt newest Friday the 13th Christmas Designs 3d print fashion long sleeve teenagers casual home pajamas dress Y10 [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    torch.zeros_like(proj_img_embedding.unsqueeze(1)),
    model.transformer.get_input_embeddings()(input_ids)
)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        inputs_embeds=embs_swapped, attention_mask=attention_mask,
        length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()

(["women's clothing > matching sets > skirt sets",
  'mother & kids > pregnancy & maternity > maternity clothing > sleep & lounge',
  "women's clothing > underwear & sleepwears > women's sleepwears > sleep tops"],
 tensor([0.5756, 0.1099, 0.0932], device='cuda:0'),
 tensor(0.7787, device='cuda:0'))

In [28]:
from torch import nn

In [29]:
# multimodal search by embedding (not working now, need discussion on loss function)
inputs = model.tokenizer("Embed image: [title start] Clothes [title end] [image start] <extra_id_99> [image end]", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

embs_swapped = torch.where(
    (input_ids == model.local_additional_special_tokens_ids_list[-1]).unsqueeze(-1), 
    proj_img_embedding.unsqueeze(1),
    model.transformer.get_input_embeddings()(input_ids)
)

attention_mask = inputs['attention_mask'].to(model.device)
hidden_states_product = model.get_hidden_states(
    input_ids = input_ids.to(model.device), 
    attention_mask = attention_mask.to(model.device),
    inputs_embeds = embs_swapped 
)

qs = ['black clothes', 'pink clothes', 'white clothes']
for q in qs:
    print(q)
    inputs = model.tokenizer(f"Embed query: {q}", return_tensors='pt')
    hidden_states_q = model.get_hidden_states(
        input_ids = inputs['input_ids'].to(model.device), 
        attention_mask = inputs['attention_mask'].to(model.device)
    )
    print(nn.functional.normalize(hidden_states_q).mm(nn.functional.normalize(hidden_states_product).T).item())
    print('-' * 20)

black clothes
0.18413197994232178
--------------------
pink clothes
0.23975355923175812
--------------------
white clothes
0.2434498518705368
--------------------


In [30]:
from copy import deepcopy
from tqdm import tqdm

In [31]:
# seqclf-based categorize query
label_map_rev = {label_map[i]: i for i in label_map}

inputs = model.tokenizer("Classify query: pillow", return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
hidden_states = model.get_hidden_states(
    input_ids = input_ids.to(model.device), 
    attention_mask = attention_mask.to(model.device)
)
logits = model.clf_head(hidden_states)
prob_nums = logits.sigmoid()

In [32]:
probs = prob_nums

In [33]:
top_probs, top_pred_indices = probs.topk(probs.size(1))
assert prediction.size(1) == len(label_map)

[(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)][:10]

[('home & garden > home textile', 0.8359919190406799),
 ('home & garden', 0.6865457892417908),
 ('home & garden > home textile > bedding', 0.38045328855514526),
 ('home & garden > home textile > bedding > pillow cases',
  0.24303345382213593),
 ('home & garden > home textile > pillows', 0.06814387440681458),
 ('home & garden > home textile > pillows > decorative pillows',
  0.031411249190568924),
 ('automobiles & motorcycles > interior accessories > neck pillow',
  0.022724634036421776),
 ('toys & hobbies > stuffed animals & plush toys > plush pillows',
  0.020035220310091972),
 ('automobiles & motorcycles', 0.01586918905377388),
 ('home & garden > home textile > table & sofa linens > cushion covers',
  0.015179386362433434)]

In [34]:
# clm-based categorize query
inputs = model.tokenizer("Generate taxonomy for query: pillow", return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
attention_mask = inputs['attention_mask'].to(model.device)
outs = model.transformer.generate(
        input_ids=input_ids, attention_mask=attention_mask,
        length_penalty=0., max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
        output_scores=True, return_dict_in_generate=True)
model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), outs.sequences_scores.exp(), outs.sequences_scores.exp().sum()

(['home & garden > home textile > pillows > decorative pillows',
  'home & garden > home textile > bedding > pillow cases',
  'home & garden > home textile > table & sofa linens > cushion'],
 tensor([0.4519, 0.1555, 0.0466], device='cuda:0'),
 tensor(0.6540, device='cuda:0'))

# baseline for query classify

In [42]:
df = pd.read_json(dvc.api.get_url(
    'data/query/wish_queries_inferred_newtax.json',
    repo='git@github.com:junwang-wish/query_understanding_data.git'
), lines=True)

In [43]:
df_tax = pd.read_json(dvc.api.get_url(
    'data/taxonomy/wish_newtax.json',
    repo='git@github.com:junwang-wish/query_understanding_data.git'
), lines=True)

In [44]:
label_map_official = {}
for i in df_tax.to_dict('records'):
    label_map_official[int(i['id'])] = i['category_path']

In [45]:
df['category_paths'] = df['categories'].apply(lambda x: [label_map_official[int(i)].lower().strip() for i in x.split(',')])

In [46]:
df.sample(2).to_dict('records')

[{'query': 'cheap womens shirts size small',
  'norm': 1.0392304659,
  'categories': '6058,6057,6023',
  'category_names': "Women's Tops,Muslim Fashion,Women's Clothing",
  'weights': '1.9245008973,0.38490017946,0.38490017946',
  'category_paths': ["women's clothing > muslim fashion > women's tops",
   "women's clothing > muslim fashion",
   "women's clothing"]},
 {'query': 'grabadora voz',
  'norm': 1.0392304659,
  'categories': '1692,1689,1495',
  'category_names': 'Digital Voice Recorder,Portable Audio & Video,Consumer Electronics',
  'weights': '1.9245008973,0.38490017946,0.38490017946',
  'category_paths': ['consumer electronics > portable audio & video > digital voice recorder',
   'consumer electronics > portable audio & video',
   'consumer electronics']}]

In [47]:
def clm_query(q):
    # clm-based categorize query
    inputs = model.tokenizer(f"Generate taxonomy for query: {q}", return_tensors='pt')
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)
    outs = model.transformer.generate(
            input_ids=input_ids, attention_mask=attention_mask,
            length_penalty=0., max_new_tokens=50, num_beams=3 ,num_return_sequences=3,
            output_scores=True, return_dict_in_generate=True)
    return model.tokenizer.batch_decode(outs.sequences, skip_special_tokens=True), \
        outs.sequences_scores.exp().detach().cpu().numpy(), \
            outs.sequences_scores.exp().sum().detach().cpu().numpy()

In [75]:
def seqclf_query(q):
    # seqclf-based categorize query
    inputs = model.tokenizer(f"Classify query: {q}", return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    hidden_states = model.get_hidden_states(
        input_ids = input_ids.to(model.device), 
        attention_mask = attention_mask.to(model.device)
    )
    logits = model.clf_head(hidden_states)
    probs = logits.sigmoid()
    top_probs, top_pred_indices = probs.topk(probs.size(1))
    assert prediction.size(1) == len(label_map)

    return [(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)][:10]

In [48]:
df_tmp = df.sample(20, random_state=42)

In [49]:
df_tmp['clm_query'] = df_tmp['query'].apply(clm_query)

In [72]:
# df_tmp[['query', 'category_paths', 'clm_query']].to_dict('records')

# question

1. To beat query baseline (a strong one since it is based directly on noisy Wish customer implicit feedback), we should exploit such implicit feedback too, but not get affected by the noise
2. For seqclf, is there a better decoding method than picking top-K predicted taxonomy paths (could be L1, L2, ...)? For example, add up the logits to leaf or a certain level with logit > lambda
3. Similarly for clm, is there a better decoding method than beam search?

Top reasons T5 model is bad at query classification
1. Non-English query --> Translate
2. Miss-spelling in query --> Data augmentation?
3. Query is "X for Y", but model classifies Y's category instead --> More labelled data for training
4. Query is "adj. + X", which changes X's category, but model gets fixated on X's category --> More labelled data for training
5. Isoteric brand name --> More labelled data for training
6. Consumer electrical / hand tools --> More labelled data for training

In [84]:
q = '3090 gpu'
clm_query(q), seqclf_query(q)

((['computer & office > servers',
   'computer & office > computer components > motherboards',
   'computer & office > computer components > internal hard drives'],
  array([0.12470548, 0.12196591, 0.07991078], dtype=float32),
  array(0.32658216, dtype=float32)),
 [('computer & office', 0.5845221281051636),
  ('home improvement', 0.296642929315567),
  ('automobiles & motorcycles', 0.1690264493227005),
  ('computer & office > computer components', 0.11630456149578094),
  ('home improvement > hardware', 0.053344424813985825),
  ('automobiles & motorcycles > auto replacement parts', 0.04588837921619415),
  ('home improvement > electrical equipments & supplies', 0.01563027687370777),
  ('automobiles & motorcycles > auto replacement parts > cooling system',
   0.014263961464166641),
  ('automobiles & motorcycles > auto replacement parts > engines & components',
   0.01367215532809496),
  ('computer & office > laptop parts', 0.012038576416671276)])

In [80]:
model.tokenizer.tokenize(q)

['▁', 'מטען', '▁', 'למברג', '▁bo', 'sch', '▁18', 'v']

In [51]:
# really bad mistakes
bad = ['jesus rhinestone hat', 'activia t', 'accessori gabbie criceti',
'מטען למברג bosch 18v', 'pince à dénuder', 'iphnoe 14', 'aparelho auditivo generic',
'mercedes e klasse 2004auße spigel', 'valkoinen kauluspaita'
]
# really good corrections
good = ['panello solare ta ventola']

In [52]:
df_tmp[df_tmp['query'].apply(lambda x: x in bad)][['query', 'category_paths', 'clm_query']].to_dict('records')

[{'query': 'jesus rhinestone hat',
  'category_paths': ["apparel accessories > women's hats > women's baseball caps",
   "apparel accessories > women's hats",
   'apparel accessories'],
  'clm_query': (["apparel accessories > women's hats > women's fedoras",
    "apparel accessories > men's hats > men's fedoras",
    "apparel accessories > women's hats > women's baseball caps"],
   array([0.41313443, 0.12854807, 0.10324986], dtype=float32),
   array(0.6449324, dtype=float32))},
 {'query': 'activia t',
  'category_paths': ['home & garden > garden supplies > pest control > repellents',
   'home improvement > lights & lighting > portable lighting > flashlights & torches',
   'home & garden > garden supplies > pest control',
   'home improvement'],
  'clm_query': (['education & office supplies > books & magazines > humanities & social science',
    'education & office supplies > books & magazines > literature & fiction',
    'automobiles & motorcycles > car wash & maintenance > paint care 